In [1]:
import os

import pandas as pd
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta

from pandas_datareader import data as pdr

start = datetime.datetime.today() - relativedelta(years=5)
end = datetime.datetime.today()

stock_list = ['^GSPC','^DJI','^VIX']
yf.pdr_override()
df_all = []
for i in stock_list:
    df_all.append(pdr.get_data_yahoo(i, start=start, end=end))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [2]:
df_join = df_all[0]
for i in range(1, len(df_all)):
    df_all[i] = df_all[i][['Close']]
    df_all[i] = df_all[i].rename({'Close': stock_list[i][1:]}, axis='columns')
    df_join = df_join.join(df_all[i])
df_join['t_Close'] = df_join.loc[:, 'Close']
shift_col = ['Open','High','Low','Close','DJI','VIX']
for j in shift_col:
    df_join[j] = df_join[j].shift(1)
    if j == 'DJI':
        df_join = df_join.rename({j: 'y_Dow'}, axis='columns')
    elif j == 'VIX':
        df_join = df_join.rename({j: 'y_' + j}, axis='columns')
    else:
        df_join = df_join.rename({j: 'y_' + j}, axis='columns')
df_join = df_join.rename({'t_Close': 'Close'}, axis='columns')
df_join = df_join.dropna()
df_join = df_join.loc['2017-01-01':'2020-07-17']

In [3]:
df_join

,y_Open,y_High,y_Low,y_Close,Adj Close,Volume,y_Dow,y_VIX,Close
Date,,,,,,,,,
2017-01-03,2251.610107,2253.580078,2233.620117,2238.830078,2257.830078,3770530000,19762.599609,14.040000,2257.830078
2017-01-04,2251.570068,2263.879883,2245.129883,2257.830078,2270.750000,3764890000,19881.759766,12.850000,2270.750000
2017-01-05,2261.600098,2272.820068,2261.600098,2270.750000,2269.000000,3761820000,19942.160156,11.850000,2269.000000
2017-01-06,2268.179932,2271.500000,2260.449951,2269.000000,2276.979980,3339890000,19899.289062,11.670000,2276.979980
2017-01-09,2271.139893,2282.100098,2264.060059,2276.979980,2268.899902,3217610000,19963.800781,11.320000,2268.899902
...,...,...,...,...,...,...,...,...,...
2020-07-13,3152.469971,3186.820068,3136.219971,3185.040039,3155.219971,4890780000,26075.300781,27.290001,3155.219971
2020-07-14,3205.080078,3235.320068,3149.429932,3155.219971,3197.520020,4476170000,26085.800781,32.189999,3197.520020
2020-07-15,3141.110107,3200.949951,3127.659912,3197.520020,3226.560059,4669760000,26642.589844,29.520000,3226.560059


In [4]:
df_join.to_csv('GSPC.csv')